In [34]:
#neural network section
import numpy as np

class NeuralNetwork:
    #initialize weights and biases randomly
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.01):
        self.weights_input_hidden = np.random.rand(input_size, hidden_size) - 0.5
        self.weights_hidden_output = np.random.rand(hidden_size, output_size) - 0.5
        self.bias_hidden = np.random.rand(hidden_size) - 0.5
        self.bias_output = np.random.rand(output_size) - 0.5
        self.learning_rate = learning_rate

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def forward(self, x):
        self.hidden_input = np.dot(x, self.weights_input_hidden) + self.bias_hidden
        self.hidden_output = self.sigmoid(self.hidden_input) 
        self.final_input = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output
        self.final_output = self.sigmoid(self.final_input)
        return self.final_output

    def backward(self, x, y, output):
        output_error = y - output
        output_delta = output_error * self.sigmoid_derivative(output)
        
        hidden_error = output_delta.dot(self.weights_hidden_output.T)
        hidden_delta = hidden_error * self.sigmoid_derivative(self.hidden_output)

        #update weights and biases
        self.weights_hidden_output += self.hidden_output.T.dot(output_delta) * self.learning_rate
        self.weights_input_hidden += x.T.dot(hidden_delta) * self.learning_rate
        self.bias_output += np.sum(output_delta, axis=0) * self.learning_rate
        self.bias_hidden += np.sum(hidden_delta, axis=0) * self.learning_rate

    def train(self, x, y, epochs=100):
        for epoch in range(epochs):
            output = self.forward(x)
            self.backward(x, y, output)

    def predict(self, x):
        return self.forward(x)


In [31]:
# dataset preprocessing section
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

def load_and_preprocess_data(filepath):
    data = pd.read_csv(filepath)
    X = data.iloc[:, :-1].values  #input features
    y = data.iloc[:, -1].values   #labels
    
    #transform text target labels into numerical ones
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)
    
    #normalize features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    #train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    return X_train, X_test, y_train, y_test


In [35]:
#implementation on iris dataset
from sklearn.metrics import accuracy_score

#load dataset
X_train, X_test, y_train, y_test = load_and_preprocess_data('iris.csv')

#one-hot encode the labels
y_train_onehot = np.eye(len(np.unique(y_train)))[y_train]
y_test_onehot = np.eye(len(np.unique(y_test)))[y_test]

#initialize and train the neural network
input_size = X_train.shape[1]
hidden_size = 4
output_size = y_train_onehot.shape[1]
learning_rate = 0.01

nn = NeuralNetwork(input_size, hidden_size, output_size, learning_rate)
nn.train(X_train, y_train_onehot, epochs=1000)

#predict on test set
y_pred = nn.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

#accuracy evaluation
accuracy = accuracy_score(y_test, y_pred_labels)
print(f"Test accuracy: {accuracy * 100:.2f}%")


Test accuracy: 97.78%
